## Features for NBaIOT

In [10]:
import os
import urllib
import zipfile
import pandas as pd
import numpy as np
import pickle
import glob
import scipy.sparse

In [11]:
def download_nbaiot():
    device_list = [
        "Danmini_Doorbell",
        "Ecobee_Thermostat",
        "Ennio_Doorbell",
        "Philips_B120N10_Baby_Monitor",
        "Provision_PT_737E_Security_Camera",
        "Provision_PT_838_Security_Camera",
        "Samsung_SNH_1011_N_Webcam",
        "SimpleHome_XCS7_1002_WHT_Security_Camera",
        "SimpleHome_XCS7_1003_WHT_Security_Camera",
    ]
    for d in device_list:
        url_root = "https://fediot.s3.us-west-1.amazonaws.com/fediot"
        url = os.path.join(url_root, (d + ".zip"))
        saved_path = os.path.join("/data/thomas/Principals/testcases/FL4IOT/data/NBaIOT", (d + ".zip"))
        urllib.request.urlretrieve(url, saved_path)
        with zipfile.ZipFile(saved_path, "r") as f:
            f.extractall("/data/thomas/Principals/testcases/FL4IOT/data/NBaIOT")

In [12]:
def get_data_nbaiot():
    device_list = [
        "Danmini_Doorbell",
        "Ecobee_Thermostat",
        "Ennio_Doorbell",
        "Philips_B120N10_Baby_Monitor",
        "Provision_PT_737E_Security_Camera",
        "Provision_PT_838_Security_Camera",
        "Samsung_SNH_1011_N_Webcam",
        "SimpleHome_XCS7_1002_WHT_Security_Camera",
        "SimpleHome_XCS7_1003_WHT_Security_Camera",
    ]
    data = {}
    i = 0
    #max_dataset = np.loadtxt("/data/thomas/Principals/testcases/FL4IOT/data/NBaIOT/max_dataset.txt")
    #min_dataset = np.loadtxt("/data/thomas/Principals/testcases/FL4IOT/data/NBaIOT/min_dataset.txt")
    for d in device_list:
        benign_data = pd.read_csv(os.path.join("/data/thomas/Principals/testcases/FL4IOT/data/NBaIOT", d, "benign_traffic.csv"))
        #benign_data = benign_data[:5000]
        benign_data = np.array(benign_data)
        benign_data[np.isnan(benign_data)] = 0
        #benign_data = (benign_data - min_dataset) / (max_dataset - min_dataset)
        data[d] = benign_data
        i+=1
    return data

In [13]:
def get_astrolavos_data(directory, min_samples=1000):
    files = glob.glob(directory+"/*")
    res = {}
    for file in files:
        #print(f"Doing file {file}")
        with open(file, 'rb') as f:
            loaded_data = np.array(pickle.load(f))
        if loaded_data.shape[0] > min_samples:
            res[number] = loaded_data
            number += 1
    return res
    

In [56]:
def get_astrolavos_data2(directory):
    legend = {
        "192.168.0.21" : "RokuTV",
        "192.168.0.43" : "GoogleHome",
        "192.168.0.10" : "NestCamera",
        "192.168.0.19" : "RingDoorbell",
        "192.168.0.48" : "Roomba",
        "192.168.0.44" : "BoseSoundTouch10",
        "192.168.0.25" : "AppleTV(4thGen)",
        "192.168.0.12" : "BelkinWeMoMotionSensor"
    }
    
    files = glob.glob(directory+"/*.npz")
    res = {}
    for f in files:
        dev = legend[f.split("/")[-1].split("_")[0]]
        sparse_bigrams = scipy.sparse.load_npz(f)[:10000,:]
        res[dev] = sparse_bigrams.todense()
    return res
